In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
trash_haulers = pd.read_csv('../data/trash_hauler_report.csv')
trash_haulers

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002


In [7]:
trash_haulers.shape

(20226, 11)

In [11]:
trash_haulers.columns

Index(['Request Number', 'Date Opened', 'Request ', 'Description',
       'Incident Address', 'Zip Code', 'Trash Hauler', 'Trash Route',
       'Council District', 'State Plan X', 'State Plan Y'],
      dtype='object')

In [28]:
# This is how I am pulling just the requests from the 'Request' column that contain 'Trash - Curbside/Alley Missed Pickup'.
missed_trash_request = trash_haulers.loc[trash_haulers['Request '] == 'Trash - Curbside/Alley Missed Pickup']
missed_trash_request

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002


In [26]:
# Here I am looking for the duplicated address and only pulling the ones that have been duplicated.
missed_trash_requests = missed_trash_request.duplicated(subset='Incident Address', keep =False)
multi_missed_trash_requests = missed_trash_request[missed_trash_requests == True]
multi_missed_trash_requests

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996
12,25359,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed. Does not have access to alley.,830 Meridian St,37207.0,METRO,9207,5.0,1741744.712,675328.6661
19,25454,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,449 Westboro Dr,37209.0,RED RIVER,4201,20.0,1710615.090,661992.3307
24,25496,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed pickup,1815 WOODMONT BLVD,37215.0,RED RIVER,3305,25.0,1730532.216,648861.6319
29,25511,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,259 Sunrise Ave,37211.0,RED RIVER,4304,16.0,1750609.593,645187.6809
...,...,...,...,...,...,...,...,...,...,...,...
20213,267035,11/1/2019,Trash - Curbside/Alley Missed Pickup,"constantly missed, please pick up","1400 Dickerson Pike, Nashville, TN 37207, Unit...",37207.0,METRO,9204,5.0,1740796.199,679412.7988
20214,267060,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire street missed,"639 Heritage Dr, Nashville, TN 37115, United S...",37115.0,RED RIVER,3412,8.0,1754841.800,705326.3996
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038


In [20]:
# Here I am just checking my work to make sure there are duplicates.
find_dupes = multi_missed_trash_requests.loc[missed_trash_request['Incident Address'] == '259 Sunrise Ave']
find_dupes

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
29,25511,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,259 Sunrise Ave,37211.0,RED RIVER,4304,16.0,1750609.593,645187.6809
2555,56781,3/28/2018,Trash - Curbside/Alley Missed Pickup,Missed.,259 Sunrise Ave,37211.0,RED RIVER,4304,16.0,1750608.475,645188.9845


In [22]:
# Here I am asking just for the addresses after the first occurence since the first incident is not payable.
payable_missed_pickups = multi_missed_trash_requests.duplicated(subset='Incident Address', keep='first')
payable_missed_pickups

1        False
12       False
19       False
24       False
29       False
         ...  
20213     True
20214     True
20221     True
20223     True
20224     True
Length: 5838, dtype: bool

In [23]:
#Here I am asking for the count of missed pickups that are payable.
count_payable = payable_missed_pickups.sum()
count_payable

3611

In [24]:
# I multiplied the number of payable missed pickups by 200 since every missed pickup after the first one is a $200.00 charge.
amount_due = count_payable * 200
amount_due

722200

In [29]:
#This is the formula that I got from Madi that converts my number into dollars.
print(f"${amount_due:,.2f}")

$722,200.00
